- 2 err chưa fix
    + Kích thước câu lớn hơn 256 thì phải cắt => Nên yêu cầu dữ liệu đầu vào sẽ được xử lý và lưu lại txt file
    + Cái tagidx cho thằng padding bị sai, nhưng không ảnh hưởng đến bài toán. (chưa tìm ra lỗi đâu)

# 1. Load data

In [1]:
from transformers import AutoTokenizer
from transformers import BertForTokenClassification, AdamW, BertModel, BertConfig
import torch

from tqdm import tqdm, trange
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report

In [2]:
from utils.bert_load import process_data_for_BERT, check_label, DF_LABEL
from utils.bert_evaluate import valuation_bert_multi

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [3]:
dir_train = 'vlsp2018/train_2018.txt'
dir_valid = 'vlsp2018/val_2018.txt'
MAXLEN = 256
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False,use_fast=False)

## 1.1 BIO Label

In [4]:
BIO_tag_values = ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']
BIO_data_train = process_data_for_BERT(dir_train, tokenizer, BIO_tag_values, MAXLEN, type='BIO')
BIO_data_valid = process_data_for_BERT(dir_valid, tokenizer, BIO_tag_values, MAXLEN, type='BIO')

#### Một số attibute

In [5]:
idx = 0
BIO_data_train.X_padding[idx]
BIO_data_train.X[idx]
BIO_data_train.data[idx]
BIO_data_train.tag2idx

{'B-LOC': 0,
 'B-MISC': 1,
 'B-ORG': 2,
 'B-PER': 3,
 'I-LOC': 4,
 'I-MISC': 5,
 'I-ORG': 6,
 'I-PER': 7,
 'O': 8,
 'PAD': 9}

In [6]:
BIO_data_train.data[idx]

[('Bế', 'O'),
 ('mạc', 'O'),
 ('Hội', 'O'),
 ('nghị', 'O'),
 ('các', 'O'),
 ('quan', 'O'),
 ('chức', 'O'),
 ('cao', 'O'),
 ('cấp', 'O'),
 ('về', 'O'),
 ('Quản', 'O'),
 ('lý', 'O'),
 ('thiên', 'O'),
 ('tai', 'O'),
 ('lần', 'O'),
 ('thứ', 'O'),
 ('11', 'O')]

#### Một số method

- có 2 mode: 'traning' và 'evaluation'
- có 2 type: 'train' và 'valid'

NX: tại vì lúc chuyển tesor lúc huấn luyện và lúc đánh giá nó khác loại nên mới dùng 2 mode còn dataloader của thằng train và thằng valid cũng khác nhau ở cách nó load, một thằng ngẫu nhiên và một thằng tuần tự nên có 2 type

In [7]:
BIO_dataloader_train = BIO_data_train.Dataloader(BATCH_SIZE = 32, mode = 'evaluation', type ='train')
BIO_dataloader_valid = BIO_data_valid.Dataloader(BATCH_SIZE = 32, mode = 'evaluation', type ='valid')

In [8]:
check_label(BIO_data_train.data + BIO_data_valid.data)

{'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O'}

In [9]:
DF_LABEL(BIO_data_train.data, BIO_data_valid.data)

,O,B-MISC,I-MISC,B-LOC,I-LOC,B-ORG,I-ORG,B-PER,I-PER
0,442522,784,1707,7254,7367,5124,11114,6363,6184
1,147392,184,436,2396,2487,1884,4489,2128,2205


## 1.2 IO label

In [10]:
IO_tag_values = ['PER','LOC','ORG','MISC','O']
IO_data_train = process_data_for_BERT(dir_train, tokenizer, IO_tag_values, MAXLEN, type='IO')
IO_data_valid = process_data_for_BERT(dir_valid, tokenizer, IO_tag_values, MAXLEN, type='IO')

In [11]:
check_label(IO_data_train.data + IO_data_valid.data)

{'LOC', 'MISC', 'O', 'ORG', 'PER'}

In [12]:
DF_LABEL(IO_data_train.data, IO_data_valid.data)

,O,MISC,LOC,ORG,PER
0,442522,2491,14621,16238,12547
1,147392,620,4883,6373,4333


In [13]:
IO_dataloader_train = IO_data_train.Dataloader(BATCH_SIZE = 32, mode = 'evaluation', type ='train')
IO_dataloader_valid = IO_data_valid.Dataloader(BATCH_SIZE = 32, mode = 'evaluation', type ='valid')

In [14]:
IO_data_valid.tag2idx

{'PER': 0, 'LOC': 1, 'ORG': 2, 'MISC': 3, 'O': 4, 'PAD': 6}

# 2. Modeling

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 2.1 IO labeling

In [16]:
IO_tag2idx = IO_data_train.tag2idx
PATH_MODEL = 'model_save/2018/IO_BERT_MULTI.pt'

In [17]:
#BERT base
IO_model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(IO_tag2idx),
    output_attentions = False,
    output_hidden_states = False
)
IO_model.load_state_dict(torch.load(PATH_MODEL), strict=False)
IO_model.to(device)
print("DONE LOAD MODEL")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

DONE LOAD MODEL


In [18]:
valuation_bert_multi(IO_model,IO_dataloader_valid,IO_tag_values,device)

Progress Bar: 100%|██████████████████████████████████████████████████████████████████| 101/101 [01:47<00:00,  1.06s/it]


Validation loss: 0.06861759977827844
Validation F1-Score: 0.9865009814542686
              precision    recall  f1-score   support

         LOC     0.9268    0.9397    0.9332      5818
        MISC     0.7713    0.6555    0.7087       746
           O     0.9954    0.9929    0.9942    165159
         ORG     0.8993    0.9397    0.9190      8584
         PER     0.9546    0.9646    0.9596      6151

    accuracy                         0.9865    186458
   macro avg     0.9095    0.8985    0.9029    186458
weighted avg     0.9866    0.9865    0.9865    186458

####################


## 2.2 BIO Labeling

In [19]:
BIO_tag2idx = BIO_data_train.tag2idx
PATH_MODEL = 'model_save/2018/BIO_BERT_MULTI.pt'

In [20]:
#BERT base
BIO_model = BertForTokenClassification.from_pretrained(
    "bert-base-multilingual-cased",
    num_labels=len(BIO_tag2idx),
    output_attentions = False,
    output_hidden_states = False
)
BIO_model.load_state_dict(torch.load(PATH_MODEL), strict=False)
BIO_model.to(device)
print("DONE LOAD MODEL")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

DONE LOAD MODEL


In [21]:
valuation_bert_multi(BIO_model,BIO_dataloader_valid,BIO_tag_values,device)

Progress Bar: 100%|██████████████████████████████████████████████████████████████████| 101/101 [01:46<00:00,  1.06s/it]


Validation loss: 0.1369466267644186
Validation F1-Score: 0.9848008666831136
              precision    recall  f1-score   support

       B-LOC     0.9057    0.9053    0.9055      2715
      B-MISC     0.6844    0.6725    0.6784       229
       B-ORG     0.8471    0.9154    0.8799      2918
       B-PER     0.9412    0.9485    0.9448      2988
       I-LOC     0.9276    0.9201    0.9238      3103
      I-MISC     0.7708    0.6634    0.7131       517
       I-ORG     0.9089    0.9206    0.9147      5666
       I-PER     0.9624    0.9545    0.9584      3163
           O     0.9947    0.9934    0.9940    165159

    accuracy                         0.9848    186458
   macro avg     0.8825    0.8771    0.8792    186458
weighted avg     0.9849    0.9848    0.9848    186458

####################
